In [7]:
import data
from torch import nn, optim
import torch
from tqdm import tqdm

In [14]:
model = nn.Sequential(
    nn.Conv1d(in_channels=1, out_channels=4, kernel_size=2), 
    nn.Conv1d(in_channels=4, out_channels=4, kernel_size=2),
    nn.MaxPool1d(kernel_size=2),
    nn.Flatten(),
    nn.LazyLinear(out_features=5), nn.Sigmoid(),
    nn.LazyLinear(out_features=3), nn.Sigmoid()  
)

In [15]:
train_music_loader = data.get_data()
loss_fn = nn.MSELoss()# compute the error 
optim_fn = optim.SGD(model.parameters(),lr=0.01) # Stochastic Gradient descent
for epoch in tqdm(range(200)):
    for xb,yb in train_music_loader: # 
        ypred = model(xb) # find the output of the model
        loss = loss_fn(ypred, yb)  #Out is the ML think is the answer, yb the real answer 
        optim_fn.zero_grad() # get ready for the slope
        loss.backward()# find where is it down the slope
        optim_fn.step()# Take a step down
        
if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


100%|██████████| 200/200 [00:04<00:00, 49.56it/s]


In [ ]:
from csound_module import Csound_instrument
import torch
import os

model.eval()
csound = Csound_instrument()

os.makedirs("output", exist_ok=True)

num_generate = 10

with torch.no_grad():
    for i in range(num_generate):
        random_audio = torch.randn(1, 1, 96000)
        
        predicted_params = model(random_audio)
        
        freq = float(predicted_params[0, 0]) * 800 + 200   # 200-1000 Hz
        dur = float(predicted_params[0, 1]) * 2 + 0.5      # 0.5-2.5 s
        amp = float(predicted_params[0, 2])                # 0-1
        
        output_file = f"output/ml_generated_{i}.wav" 
        csound.csound_instrument_score(
            instr_number=1,
            frequency=freq,
            duration=dur,
            save_file=output_file
        )
        

rtaudio: PortAudio module enabled ...
using callback interface
rtmidi: PortMIDI module enabled
--Csound version 6.18 (double samples) Nov 24 2022
[commit: a1580f9cdf331c35dceb486f4231871ce0b00266]
libsndfile-1.1.0
graphics suppressed, ascii substituted
sr = 48000.0, kr = 1500.000, ksmps = 32
0dBFS level = 1.0, A4 tuning = 440.0
orch now loaded
audio buffered in 1024 sample-frame blocks
writing 4096-byte blks of shorts to  _temp_1.wav (WAV)
SECTION 1:
new alloc for instr 1:
B  0.000 ..  2.012 T  2.013 TT  2.013 M:  0.50000  0.50000
Score finished in csoundPerformKsmps() with 2.
inactive allocs returned to freespace
end of score.		   overall amps:  0.50000  0.50000
	   overall samples out of range:        0        0
0 errors in performance
Elapsed time at end of performance: real: 0.004s, CPU: 0.004s
2048 4096 sample blks of shorts written to  _temp_1.wav (WAV)
rtaudio: PortAudio module enabled ...
using callback interface
rtmidi: PortMIDI module enabled
--Csound version 6.18 (double sam